In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer as TfidfVectorizer
from joblib import dump
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

import time

In [41]:
data = pd.read_excel('data/Data_B_Bersih.xlsx')
label = data['label']
# y = [str(x) for x in label]
respon = data['respon'].tolist()
# respon
y = np.array(label)

In [42]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(respon).A
save_ = vectorizer.fit(respon)
# print(vectorizer.get_feature_names())
dump(respon, 'model/tfdf_B.joblib')

['model/tfdf_B.joblib']

In [36]:
X_train = X[:-10]
Y_train = y[:-10]

In [38]:
neigh = KNeighborsClassifier(n_neighbors=10)
P_CNB = neigh.fit(X_train, Y_train)

In [45]:
i

0

In [43]:
K = 10 #pembagian data uji dan data latih
# from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score #, precision_score, recall_score
# kf= KFold(n_splits=K) 
skf = StratifiedKFold(n_splits=K,random_state=2, shuffle=True)
skf

StratifiedKFold(n_splits=10, random_state=2, shuffle=True)

In [46]:
# K = 10 #pembagian data uji dan data latih
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score

# kf = KFold(n_splits=K) 
C_range = np.arange(1.0,5,0.1)
gamma_range = np.arange(0.1,2, 0.1)

C_range_ = []
gamma_range_ = []
rerata_akurasi_svm = []
rerata_waktu_eksekusi = []
rerata_waktu_prediksi = []
count=0
for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        C = C_range[i]
        gamma = gamma_range[j]
        akurasi_svm = []
        waktu_eksekusi = []
        waktu_prediksi = []
        cSVM = svm.SVC(kernel='rbf', gamma=gamma, C=C) #model
        for train_index, test_index in skf.split(X, y):
#             print(train_index)
            #membagi data latih dan data uji
            X_train, X_test = X[train_index], X[test_index]
#             print(train_index)
            y_train = y[train_index]
            y_test  = y[test_index]
            
            start = time.time()
            #Melatih Model
            cSVM = cSVM.fit(X_train, y_train) #SVM
            end = time.time()
            waktu_eksekusi.append(end-start)
            
            #proses prediksi
            start = time.time()
            P_SVM = cSVM.predict(X_test)
            end = time.time()
            waktu_prediksi.append(end-start)
            
            akurasi_svm.append(accuracy_score(y_test, P_SVM))
        print("iterasi ke-", count)
        C_range_.append(C)
        gamma_range_.append(gamma)
        
        rr = sum(akurasi_svm)/len(akurasi_svm)
        rerata_akurasi_svm.append(rr)
        count+=1
        
        rerata_waktu_eksekusi.append(sum(waktu_eksekusi)/len(waktu_eksekusi))
        rerata_waktu_prediksi.append(sum(waktu_prediksi)/len(waktu_prediksi))
        
        #Menampilkan hasil 
        print("-"*30)
        print("C     : ", C)
        print("gamma : ", gamma)
        print("-"*20)
        print("Rata-rata Akurasi SVM : %0.2f" % (sum(akurasi_svm)/len(akurasi_svm)))
        print("Rata-rata Waktu eksekusi SVM: %0.2f" % (sum(waktu_eksekusi)/len(waktu_eksekusi)))
        print("Rata-rata Waktu prediksi  SVM: %0.2f" % (sum(waktu_prediksi)/len(waktu_prediksi)))
        print("="*30)
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_svm)
#al = list(alpha_range)
index = rerata_akurasi_svm.index(x)
round(gamma_range_[index],2)
print("---------------------------")
print("gamma terbaik adalah :", round(gamma_range_[index],2))
print("    C terbaik adalah :", round(C_range_[index],2))
print("             Akurasi :", round(x, 2))

#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm, "waktu eksekusi":rerata_waktu_eksekusi, "waktu prediksi":rerata_waktu_prediksi}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi", "waktu eksekusi", "waktu prediksi"])
writer = pd.ExcelWriter('data/parameter/parameter-akurasi-svm_label_knn.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter')
#Close the Pandas Excel writer and output the Excel file.
writer.save()

2....................................................................................................0.5983333333333333
3....................................................................................................0.5958333333333333
4....................................................................................................0.6041666666666667
5....................................................................................................0.655
6....................................................................................................0.65
7....................................................................................................0.6458333333333331
8....................................................................................................0.6433333333333334
9....................................................................................................0.6808333333333333
10.................................................................